In [1]:
import os
os.chdir('..')

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [3]:
from src.api.common.dependency_container import DependencyContainer
from IPython.display import HTML  

DC = DependencyContainer(); DC.initialize()
pregunta='Cambié mi contraseña y no me acuerdo cuál era. ¿qué hago?'
request = {'Pregunta': pregunta, 'FAQs': faqs + faqs_unir}
response = DependencyContainer.get_schema_generator_workflow().execute(request)
HTML(response['Contenido'])

In [ ]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs  
from dash import Dash, html, dcc, Input, Output  
import dash_bootstrap_components as dbc  
from src.api.common.dependency_container import DependencyContainer  
import base64  
  
DC = DependencyContainer()  
DC.initialize()  
  
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])  
  
app.layout = dbc.Container([  
    dbc.Row([dbc.Col([html.H1("ChatBot Becat - FAQs", className="text-center mb-4")])]),  
    dbc.Row([  
        dbc.Col([  
            dcc.Loading(  
                id="loading",  
                type="default",  
                children=html.Div(id='chat_area')  
            ),  
            dcc.Input(id='user_input', type='text', placeholder='Escribe tu pregunta...', style={'width': '100%'}, debounce=True),  
            dbc.Button('Enviar', id='send_button', n_clicks=0, className="btn btn-primary")  
        ], width=12)  
    ])  
], fluid=True)  
  
@app.callback(  
    Output('chat_area', 'children'),  
    Input('send_button', 'n_clicks'),  
    Input('user_input', 'value'),  
    prevent_initial_call=True  
)  
def update_chat(n_clicks, user_input):  
    if user_input is None:  
        return ""  
    request = {'Pregunta': user_input, 'FAQs': obtener_faqs()}  
    response = DependencyContainer.get_schema_generator_workflow().execute(request)  
    encoded_content = base64.b64encode(response['Contenido'].encode('utf-8')).decode('utf-8')  
    src = f"data:text/html;base64,{encoded_content}"  
    return html.Iframe(src=src, style={'width': '100%', 'height': '500px', 'border': 'none'}, id='chat_iframe')  
  
if __name__ == '__main__':  
    app.run_server(debug=True)  